In [ ]:
# load clip model and preprocess
from clip import clip

clip_model, preprocess = clip.load("ViT-B/32", device="cuda")


# load coco_test dataset
import torch
import pickle

with open("dataset/dataset_cache/coco_test_dataset_5000.pkl", "rb") as f:
        test_dataset = pickle.load(f)

# setting of matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from utils import *

from CLIP_prefix_caption.train import *

dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
prefix_length = 10
prefix_length_clip = 40
prefix_dim = 512
num_layers = 8

ClipCap = ClipCaptionModel(prefix_length, prefix_length_clip, prefix_dim, num_layers,"mlp").to(device)
ClipCap.load_state_dict(torch.load("models/official_model/clipcap_coco_weights.pt"), strict=False)

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# generate_test(ClipCap, CLIP_Net, dataloader, tokenizer, sample_num, generate_mode = "beam", device = "cuda:0", prefix_length = 10)
for i in range(10):
        generated_text = generate_test(ClipCap, clip_model, dataloader, tokenizer, 10, generate_mode = "greedy", device = "cuda:0", prefix_length = 10, temperature=0.7)
        print(generated_text)




In [ ]:
import pickle
import torch
import matplotlib.pyplot as plt
with open("dataset/dataset_cache/coco_test_dataset_5000.pkl", "rb") as f:
    coco_test_dataset_A = pickle.load(f)
    coco_test_dataset_A.prefix_length = 10

for i in range(20,30):
    image = coco_test_dataset_A.get_img(i)
    plt.imshow(image)
    plt.axis("off")
    plt.savefig(f"coco_{i}.png")



In [ ]:
# setting of matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from utils import *

from CLIP_prefix_caption.train import *

dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
prefix_length = 10
prefix_length_clip = 40
prefix_dim = 512
num_layers = 8

ClipCap = ClipCaptionModel(prefix_length, prefix_length_clip, prefix_dim, num_layers,"mlp").to(device)
ClipCap.load_state_dict(torch.load("models/official_model/clipcap_coco_weights.pt"), strict=False)

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# generate_test(ClipCap, CLIP_Net, dataloader, tokenizer, sample_num, generate_mode = "beam", device = "cuda:0", prefix_length = 10)
generated_text = generate_test(ClipCap, clip_model, dataloader, tokenizer, 1, generate_mode = "beam", device = "cuda:0", prefix_length = 10)


# for batch in dataloader:
#     img, caption, vlm_token, gpt_token, gpt_mask, index, image_id = batch
#     img, gpt_token, gpt_mask = img.to(device), gpt_token.to(device), gpt_mask.to(device)
#     prefix = clip_model.encode_image(img).to(device, dtype=torch.float32)
#     prefix_embeds = ClipCap.clip_project(prefix).reshape(prefix.shape[0], prefix_length, -1)
#     for idx, prefix_embed in enumerate(prefix_embeds):
#         prefix_embed = prefix_embed.unsqueeze(0)    
#         generated_text = generate_beam(ClipCap, tokenizer, embed=prefix_embed, temperature=0)[0]
#         # generated_text = generate2(agent.ClipCap, agent.tokenizer, embed=prefix_embed, temperature=0.5)
#         print(generated_text)
#         print(caption[idx])
#         plt.imshow(img[idx].cpu().permute(1, 2, 0))
#         plt.show()

#     break

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset

for i in range(10):
    print(coco_train_dataset.gpt_tokens[i].shape)

print(coco_train_dataset.max_seq_len)
print(cc3m_train_dataset.max_seq_len)

max_seq_len = max(coco_train_dataset.max_seq_len, cc3m_train_dataset.max_seq_len)
coco_train_dataset.max_seq_len = max_seq_len
cc3m_train_dataset.max_seq_len = max_seq_len

coco_dataloader = DataLoader(coco_train_dataset, batch_size=64, shuffle=True)
cc3m_dataloader = DataLoader(cc3m_train_dataset, batch_size=64, shuffle=True)

coco_batch = next(iter(coco_dataloader))
cc3m_batch = next(iter(cc3m_dataloader))

print(coco_batch[0].shape, cc3m_batch[0].shape)
print(len(coco_batch[1]), len(cc3m_batch[1]))
print(coco_batch[2].shape, cc3m_batch[2].shape)
print(coco_batch[3].shape, cc3m_batch[3].shape)
print(coco_batch[4].shape, cc3m_batch[4].shape)
print(coco_batch[5].shape, cc3m_batch[5].shape)


# 二つのデータセットを結合
concat_dataset = ConcatDataset([coco_train_dataset, cc3m_train_dataset])
concat_dataset.prefix_length = coco_train_dataset.prefix_length
print(len(concat_dataset))



# 結合したデータセットのためのデータローダーを作成
data_loader = DataLoader(concat_dataset, batch_size=64, shuffle=True)

# データローダーを使ってデータを取得するループの例
for batch in data_loader:
    # batchには二つのデータセットの要素が含まれる
    # ここでデータの処理を行う
    break

print(batch[0].shape)

# save the dataset
import pickle
with open("dataset/dataset_cache/coco_cc3m_train.pkl", "wb") as f:
    pickle.dump(concat_dataset, f)

In [ ]:
with open("dataset/dataset_cache/coco_cc3m_train.pkl", "rb") as f:
    dataset = pickle.load(f)

print(len(dataset))

dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


batch = next(iter(dataloader))

print(batch[1][:10])
    

In [ ]:
from utils import *
data_path = 'dataset/'
prefix_length = 40
normalize_prefix = True
dataset = ConceptualDatasetFull(data_mode="test", transform=preprocess)

print(dataset.max_seq_len)



In [ ]:
from utils import *
data_path = 'dataset/'
prefix_length = 40
normalize_prefix = True
dataset = ConceptualDatasetFull(data_mode="train", transform=preprocess)
with open('dataset/dataset_cache/cc3m_train.pkl', 'wb') as f:
    pickle.dump(dataset, f)

In [ ]:
from utils import *

from clip import clip
data_path = 'dataset/'
prefix_length = 40
normalize_prefix = True
clip_model, preprocess = clip.load("ViT-B/32", device="cuda")
dataset = ConceptualDatasetFull(data_mode="test", transform=preprocess)
print(len(dataset))
with open("dataset/dataset_cache/conceptual_test_dataset_11467.pkl", 'wb') as f:
    pickle.dump(dataset, f)


In [ ]:
import pickle

with open("dataset/dataset_cache/cc3m_valid.pkl", "rb") as f:
    valid_dataset = pickle.load(f)

In [ ]:
import pickle

with open("dataset/dataset_cache/cc3m_train.pkl", "rb") as f:
    train_dataset = pickle.load(f)

In [ ]:
data_path = 'dataset/'
prefix_length = 40
normalize_prefix = True
dataset = ConceptualDatasetFull(data_mode="train", transform=preprocess)

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from utils import *
dataloader = DataLoader(train_dataset, batch_size=256, shuffle=False, num_workers=8, pin_memory=True)

i = 0
for batch in dataloader:
    print(i, batch[1][0])
    i+=1

In [3]:
from train_ClipCap import *
from utils import *
import pickle
from clip import clip

clip_model, preprocess = clip.load("ViT-B/32", device="cuda")


data_path = 'dataset/'
prefix_length = 10
normalize_prefix = True
dataset = CocoDataset(root = data_path, transform=preprocess, prefix_length=prefix_length, normalize_prefix=normalize_prefix, datasize="50000", data_mode="train", use_imageid=True)
print("Dataset size: ", len(dataset))

# set dataset ids [10000, ..., 20000]
dataset.ids = dataset.ids[10000:20000]
dataset.vlm_tokens, dataset.gpt_tokens, dataset.captions = dataset.get_tokens_list()
print("Dataset size: ", len(dataset))
print(dataset.captions[:10])


# save the dataset
with open('dataset/dataset_cache/coco_train_dataset_10000_20000_imageid.pkl', 'wb') as f:
    pickle.dump(dataset, f)

creating index...
index created!
Dataset size:  50000
Dataset size:  10000
['A young woman is riding a surfboard in the ocean.', 'He is about to bite into his first piece of pizza.', 'A piece of paper that has the word love written all over it.', 'a living room with a couch, television and a window', 'a close up of a green window frame', 'A man taking a swing at a tennis ball', 'Two tennis players extend their arms to shake hands with their opponents.', 'Electronic devices are underneath a flat screen television.', 'A picture of the TV on top of a shelf.', 'A woman eating a pizza and smiling for the camera.']


In [ ]:
# load clip model and preprocess
from clip import clip

clip_model, preprocess = clip.load("ViT-B/32", device="cuda")

from train_ClipCap import *
from utils import *
import pickle
data_path = 'dataset/'
prefix_length = 10
normalize_prefix = True
dataset = CocoDataset(root = data_path, transform=preprocess, prefix_length=prefix_length, normalize_prefix=normalize_prefix, data_mode="test", datasize="10000", use_imageid=True)
print("Dataset size: ", len(dataset))

# save the dataset
with open(f'dataset/dataset_cache/coco_test_dataset_{len(dataset)}.pkl', 'wb') as f:
    pickle.dump(dataset, f)

In [ ]:
from train_ClipCap import *
from utils import *
import pickle
data_path = 'dataset/'
prefix_length = 40
normalize_prefix = True
clip_model, preprocess = clip.load("ViT-B/32", device="cuda:0")
# dataset = CocoDataset(root = data_path, transform=preprocess, prefix_length=prefix_length, normalize_prefix=normalize_prefix, datasize="full_100")
# print("Dataset size: ", len(dataset))
# # save the dataset
# with open('dataset/dataset_cache/coco_train_dataset_100.pkl', 'wb') as f:
#     pickle.dump(dataset, f)


test_dataset = CocoDataset(root = data_path, transform=preprocess, prefix_length=prefix_length, normalize_prefix=normalize_prefix, data_mode="test", datasize="full", use_imageid=True)
print("Dataset size: ", len(test_dataset))

with open('dataset/dataset_cache/coco_test_dataset.pkl', 'wb') as f:
    pickle.dump(test_dataset, f)

In [ ]:
# load clip model and preprocess
from clip import clip

clip_model, preprocess = clip.load("ViT-B/32", device="cuda:1")

datasize = 300

from utils import *
data_path = 'dataset/conceptual_captions_subset/'
prefix_length = 40
normalize_prefix = True
# conceptual_dataset = ConceptualDatasetFull(data_mode="train", transform=preprocess, prefix_length=prefix_length, normalize_prefix=normalize_prefix, datasize=f"full_{datasize}")

# with open(f'dataset/dataset_cache/conceptual_train_dataset_{datasize}.pkl', 'wb') as f:
#     pickle.dump(conceptual_dataset, f)

test_dataset = ConceptualDatasetFull(data_mode="test", transform=preprocess, prefix_length=prefix_length, normalize_prefix=normalize_prefix, datasize=f"full_{datasize}")
print("Dataset size: ", len(test_dataset))

with open(f'dataset/dataset_cache/conceptual_test_dataset_{len(test_dataset)}.pkl', 'wb') as f:
    pickle.dump(test_dataset, f)


In [ ]:
print("Dataset size: ", len(conceptual_dataset))
# create the dataloader
conceptual_dataloader = DataLoader(conceptual_dataset, batch_size=32, shuffle=True, num_workers=4)
batch = next(iter(conceptual_dataloader))
print(batch[0].shape)

# save the dataset
with open('dataset/dataset_cache/conceptual_train_dataset_100.pkl', 'wb') as f:
    pickle.dump(conceptual_dataset, f)

In [1]:
# load clip model and preprocess
from clip import clip

_, preprocess = clip.load("ViT-B/32", device="cpu")

from train_ClipCap import *
from utils import *
import pickle
data_path = 'dataset/'
prefix_length = 10
normalize_prefix = True
datasize = "5000"
use_imageid = True
dataset = CocoDataset(root = data_path,data_mode="test", transform=preprocess, prefix_length=prefix_length, normalize_prefix=normalize_prefix, datasize=datasize, use_imageid=use_imageid)

# save the dataset
save_path = f'dataset/dataset_cache/coco_test_dataset_{datasize}'
if use_imageid:
    save_path += "_imageid"
with open(f'{save_path}.pkl', 'wb') as f:
    pickle.dump(dataset, f)



/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loading annotations into memory...
Done (t=0.27s)
creating index...
index created!


: 

In [ ]:
# print(dataloader.dataset.vlm_tokens)
print(dataloader.dataset.vlm_tokens[[0,1,2,3,4,5,6,7,8,9]])
vlm_tokens = torch.cat(dataloader.dataset.vlm_tokens, dim=0).reshape(len(dataloader.dataset), -1)
print(vlm_tokens.shape)
print(vlm_tokens[0])


In [ ]:
# from ProbVLM.src.networks import *
# from time import time
# ProbVLM_Net = BayesCap_for_CLIP(inp_dim=512, out_dim=512, hid_dim=256, num_layers=3, p_drop=0.05,)
# ProbVLM_Net.load_state_dict(torch.load("exp/debug/probvlm-022.pth"))
# batch = next(iter(dataloader))

# img = batch[0].cuda().float()
# cap = batch[1]
# vlm_token = batch[2]
# gpt_token = batch[3]
# print(vlm_token[0])
# print(gpt_token[0])


# print(cap[:2])

# dataloader.dataset.caption_list[0] = "yuki chan"
# dataloader.dataset.caption_list[1] = "ayano chan"

# dataloader.dataset.gpt_tokens[0] = torch.tensor(dataloader.dataset.gpt_tokenizer.encode("yuki chan"))
# dataloader.dataset.gpt_tokens[1] = torch.tensor(dataloader.dataset.gpt_tokenizer.encode("ayano chan"))
# dataloader.dataset.vlm_tokens[0] = dataloader.dataset.vlm_tokenizer("yuki chan")
# dataloader.dataset.vlm_tokens[1] = dataloader.dataset.vlm_tokenizer("ayano chan")


# print(torch.tensor(dataloader.dataset.gpt_tokenizer.encode("yuki chan")))
# print(dataloader.dataset.vlm_tokenizer("yuki chan"))

# batch = next(iter(dataloader))

# img = batch[0].cuda().float()
# cap = batch[1]
# token = batch[2]

# print(cap[:2])
# print(token[:2])
# print(batch[3][:2])